In [3]:
import os
import pandas as pd
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [51]:
# Define input and output directories
input_dir = 'G:/Official_Vanda_Organizing/Spatial_Morphologic_Data/2_Data_Cleaning/3_Extracted_Maxima_and_Heights/Maxima_Points_CSVs'  # Replace with the path to your CSV files
output_dir = 'G:/Official_Vanda_Organizing/Spatial_Morphologic_Data/3_Statistics/Voronoi_Tesselations'  # Replace with the path to save Voronoi plots

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Voronoi tessellations have been processed and saved.


In [ ]:
# Function to create Voronoi tessellation with a dynamic bounding box
def create_voronoi_tessellation_with_styling(file_path, output_file_path, buffer=1):
    # Read CSV file
    data = pd.read_csv(file_path)
    
    # Ensure the file has X and Y columns
    if 'X' not in data.columns or 'Y' not in data.columns:
        print(f"Error: Missing X or Y columns in {file_path}.")
        return
    
    points = data[['X', 'Y']].values  # Extract XY data
    
    # Check if there are enough points for Voronoi tessellation
    if len(points) < 3:
        print(f"Error: Not enough points to create a Voronoi diagram in {file_path}.")
        return
    
    # Calculate the bounding box with buffer
    min_x, min_y = points.min(axis=0)
    max_x, max_y = points.max(axis=0)
    range_x = max_x - min_x
    range_y = max_y - min_y
    min_x -= buffer * range_x
    max_x += buffer * range_x
    min_y -= buffer * range_y
    max_y += buffer * range_y
    
    # Define the bounding box as a Shapely Polygon
    bounds = [[min_x, min_y], [max_x, min_y], [max_x, max_y], [min_x, max_y]]
    bounding_box = Polygon(bounds)
    
    # Generate Voronoi tessellation
    vor = Voronoi(points)
    
    # Clip Voronoi regions to the bounding box
    def clip_region(region):
        if -1 not in region:  # Ignore infinite regions
            poly = Polygon([vor.vertices[i] for i in region])
            return poly.intersection(bounding_box)
        return None

    regions = [clip_region(region) for region in vor.regions if len(region) > 0]
    
    # Plot the bounded Voronoi diagram
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Plot Voronoi cells
    for region in regions:
        if region and not region.is_empty:  # Ignore None and empty polygons
            x, y = region.exterior.xy
            ax.plot(x, y, color='black', linewidth=0.8)  # Black edges
            ax.fill(x, y, color='white')  # White fill
    
    # Plot points
    ax.plot(points[:, 0], points[:, 1], 'o', color='navy', markersize=4)  # Slightly smaller points
    
    # Set plot title and labels
    ax.set_title(f"Voronoi Tessellation: {os.path.basename(file_path)}")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    
    # Set the axis limits to match the bounding box
    ax.set_xlim(min_x, max_x)
    ax.set_ylim(min_y, max_y)
    
    # Fix the aspect ratio to ensure the bounding box and plot limits match
    ax.set_aspect('equal', adjustable='box')
    
    # You can comment or remove the following line to make the bounding box invisible
    # ax.plot(*zip(*bounds, bounds[0]), color='white', linewidth=1)  # Bounding box line color is set to white
    
    # Save the plot to the output directory
    fig.savefig(output_file_path, bbox_inches='tight')
    plt.close(fig)  # Close the plot to avoid displaying it during batch processing

In [ ]:
# Loop through each CSV file in the input directory and process it
for file_name in os.listdir(input_dir):
    if file_name.endswith('.csv'):  # Process only CSV files
        input_file_path = os.path.join(input_dir, file_name)
        output_file_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_voronoi.png")
        
        # Call the function for each CSV file
        create_voronoi_tessellation_with_styling(input_file_path, output_file_path, buffer=0.0)

print("Voronoi tessellations have been processed and saved.")